In [13]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
def preprocess_image(image, output_image_path):
    # Size of the trained model
    new_size = (2048, 1024)

    # Calculate the aspect ratio of the image
    original_ratio = image.size[0] / image.size[1]
    new_ratio = new_size[0] / new_size[1]

    # Resize the image to fit within the new dimensions while preserving aspect ratio and crop if necessary
    if original_ratio >= new_ratio:
        temp_height = new_size[1]
        temp_width = round(new_size[1] * original_ratio)
    else:
        temp_width = new_size[0]
        temp_height = round(new_size[0] / original_ratio)
        
    temp_size = (temp_width, temp_height)
    resized_image = image.resize(temp_size, Image.ANTIALIAS)
    
    # Calculate the cropping area
    left = (resized_image.width - new_size[0]) / 2
    top = (resized_image.height - new_size[1]) / 2
    right = (resized_image.width + new_size[0]) / 2
    bottom = (resized_image.height + new_size[1]) / 2

    # Crop the image
    resized_image = resized_image.crop((left, top, right, bottom))

    # Convert the image to greyscale
    resized_image = resized_image.convert("L")

    # Resize image to 512 x 512
    resized_image = resized_image.resize((512, 512), Image.ANTIALIAS)

    # Save the result
    resized_image.save(output_image_path, format="PNG")

def preprocess_images_and_masks(input_images_folder, input_masks_folder, output_dir_images, output_dir_masks):
    # Create output directories if they don't exist
    os.makedirs(output_dir_images, exist_ok=True)
    os.makedirs(output_dir_masks, exist_ok=True)

    image_files = sorted(os.listdir(input_images_folder))
    mask_files = sorted(os.listdir(input_masks_folder))

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(input_images_folder, img_file)
        mask_path = os.path.join(input_masks_folder, mask_file)

        # Open images
        image = Image.open(img_path)
        mask = Image.open(mask_path)

        # Preprocess images and masks
        preprocess_image(image, os.path.join(output_dir_images, f"{os.path.splitext(img_file)[0]}_512x512_grey.png"))
        preprocess_image(mask, os.path.join(output_dir_masks, f"{os.path.splitext(mask_file)[0]}_512x512_grey.png"))

# Input and output directories
input_images_folder = 'images'
input_masks_folder = 'masks'
output_dir_images = 'images_resized'
output_dir_masks = 'masks_resized'

# Preprocess images and masks
preprocess_images_and_masks(input_images_folder, input_masks_folder, output_dir_images, output_dir_masks)


/var/folders/6b/xtl7vy6938n0dnfml7qyq69h0000gn/T/ipykernel_86602/204373724.py:22: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize(temp_size, Image.ANTIALIAS)
/var/folders/6b/xtl7vy6938n0dnfml7qyq69h0000gn/T/ipykernel_86602/204373724.py:37: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = resized_image.resize((512, 512), Image.ANTIALIAS)


In [10]:
def load_image_and_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)  # Assuming grayscale images
    image = tf.image.resize(image, (512, 512))  # Resize if needed
    image = image / 255.0  # Normalize to [0, 1]

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # Assuming grayscale masks
    mask = tf.image.resize(mask, (512, 512))  # Resize if needed
    mask = mask / 255.0  # Normalize to [0, 1]

    return image, mask

def load_images_and_masks(image_dir, mask_dir, subset_size=None):
    image_paths = sorted([os.path.join(image_dir, file) for file in os.listdir(image_dir)])[:subset_size]
    mask_paths = sorted([os.path.join(mask_dir, file) for file in os.listdir(mask_dir)])[:subset_size]

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
    dataset = dataset.map(load_image_and_mask, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

# Path to your images and masks directories
image_directory = 'images_resized'
mask_directory = 'masks_resized'

# Load a subset of images and masks into a TensorFlow dataset
subset_size = 10  # Define the subset size
images_and_masks_subset = load_images_and_masks(image_directory, mask_directory, subset_size=subset_size)

2023-11-16 10:09:20.624681: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [10]
	 [[{{node Placeholder/_1}}]]


In [14]:
# Split Data
def split_dataset(dataset, train_size=0.8, shuffle=True):
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    if shuffle:
        dataset = dataset.shuffle(buffer_size=dataset_size)
    train_dataset = dataset.take(int(train_size * dataset_size))
    val_dataset = dataset.skip(int(train_size * dataset_size))
    return train_dataset, val_dataset

train_dataset, val_dataset = split_dataset(images_and_masks_subset)

# U-Net Model
def unet_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)

    # Decoder
    up4 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv3)
    merge4 = layers.concatenate([up4, conv2], axis=3)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge4)

    up5 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv4)
    merge5 = layers.concatenate([up5, conv1], axis=3)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge5)

    # Output layer
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Batch Size
batch_size = 4
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

# Instantiate the U-Net model
input_shape = (512, 512, 1)  # Input shape for grayscale images
model = unet_model(input_shape)

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Training the model
model.fit(train_dataset, 
          epochs=10, 
          validation_data=val_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

Epoch 1/10


2023-11-16 10:13:43.099790: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-11-16 10:13:43.100015: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [10]
	 [[{{node Placeholder/_1}}]]


2/2 [==============================] - ETA: 0s - loss: 0.6682 - accuracy: 0.2671

2023-11-16 10:13:50.156879: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-11-16 10:13:50.157047: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [10]
	 [[{{node Placeholder/_0}}]]


2/2 [==============================] - 8s 4s/step - loss: 0.6682 - accuracy: 0.2671 - val_loss: 0.5664 - val_accuracy: 0.4587
Epoch 2/10
2/2 [==============================] - 7s 4s/step - loss: 0.5049 - accuracy: 0.3681 - val_loss: 0.2651 - val_accuracy: 0.3595
Epoch 3/10
2/2 [==============================] - 7s 4s/step - loss: 0.2320 - accuracy: 0.3429 - val_loss: 0.3040 - val_accuracy: 0.3668
Epoch 4/10
2/2 [==============================] - 7s 4s/step - loss: 0.2724 - accuracy: 0.3429 - val_loss: 0.2732 - val_accuracy: 0.4671
Epoch 5/10
2/2 [==============================] - 7s 4s/step - loss: 0.2909 - accuracy: 0.3582 - val_loss: 0.1636 - val_accuracy: 0.3648
Epoch 6/10
2/2 [==============================] - 7s 4s/step - loss: 0.1982 - accuracy: 0.3676 - val_loss: 0.2234 - val_accuracy: 0.3732
Epoch 7/10
2/2 [==============================] - 7s 4s/step - loss: 0.2465 - accuracy: 0.3586 - val_loss: 0.2380 - val_accuracy: 0.4241
Epoch 8/10
2/2 [==============================] - 8s

In [22]:
from PIL import Image

# Function to load and preprocess the image
def load_and_preprocess_image(image_path, input_shape):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)
    img = tf.image.resize(img, input_shape)
    img = img / 255.0
    img = tf.expand_dims(img, axis=0)  # Add an extra dimension for the batch size
    return img

# Function to save the mask as an image
def save_mask_image(mask, save_path):
    # Postprocess the predicted mask
    mask = tf.squeeze(mask, axis=0)  # Remove the batch size dimension
    mask = (mask > 0.5)  # Thresholding the mask
    
    # Ensure we have a channel dimension
    if len(mask.shape) == 2:
        mask = mask[..., tf.newaxis]

    # Save the mask as an image
    mask_img = tf.keras.preprocessing.image.array_to_img(mask)
    mask_img.save(save_path)

# Load an example image
image_path = '/Users/severin/Documents/GitHub/u-net-segmentation-of-streets-and-cars/images_resized/zurich_000004_000019_leftImg8bit_512x512_grey.png'  # Replace with your image path
img = load_and_preprocess_image(image_path, input_shape=(512, 512))

# Predict the mask using the trained model
pred_mask = model.predict(img)

# Save the predicted mask as an image
mask_save_path = 'predicted_mask.png'  # Replace with your save path
save_mask_image(pred_mask, mask_save_path)

1/1 [==============================] - 0s 277ms/step
